In [1]:
from cleaning import data_new, X_train, X_test, y_train, y_test

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Assuming X and y are your data
# X, y = ...

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define categorical and numerical columns
categorical_cols = X_train.select_dtypes(include='object').columns
numerical_cols = X_train.select_dtypes(include=np.number).columns

# Define preprocessing pipelines
categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline([
    ('scaler', StandardScaler())
])

# Combine preprocessors using ColumnTransformer
preprocessor = ColumnTransformer([
    ('cat', categorical_transformer, categorical_cols),
    ('num', numerical_transformer, numerical_cols)
])
# Apply preprocessing
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Convert y_train and y_test to integers using LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
# Create a logistic regression model
log_reg_model = LogisticRegression(random_state=42)

# Use Recursive Feature Elimination (RFE) with logistic regression
rfe = RFE(estimator=log_reg_model, n_features_to_select=10)  # Adjust n_features_to_select as needed

# Create a model pipeline with preprocessing and RFE
model_pipeline_rfe = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', rfe),
    ('model', log_reg_model)
])

# Train the model
model_pipeline_rfe.fit(X_train, y_train)

# Predict on test data
y_pred_rfe = model_pipeline_rfe.predict(X_test)

# Calculate accuracy and print classification report
accuracy_rfe = accuracy_score(y_test, y_pred_rfe)
print("Logistic Regression Model (RFE Feature Selection)")
print("Accuracy:", accuracy_rfe)
print(classification_report(y_test, y_pred_rfe))


/home/harsha/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

Logistic Regression Model (RFE Feature Selection)
Accuracy: 0.5702479338842975
              precision    recall  f1-score   support

           0       0.57      0.99      0.72       136
           1       0.75      0.03      0.05       106

    accuracy                           0.57       242
   macro avg       0.66      0.51      0.39       242
weighted avg       0.65      0.57      0.43       242



In [6]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

# Assuming X_train, y_train, X_test, and y_test are your data
# Define numeric and categorical features
numeric_features = X.select_dtypes(include='number').columns
categorical_features = X.select_dtypes(include='object').columns

# Define preprocessors
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Define a logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)

# Define Recursive Feature Elimination (RFE) with backward selection
rfe = RFE(estimator=model, n_features_to_select=None, step=1)  # Start with all features

# Create a pipeline with the preprocessor, RFE, and Logistic Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfe', rfe),
    ('clf', model)
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print accuracy and classification report
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

# Get the transformed feature names after preprocessing
feature_names = np.concatenate([
    preprocessor.named_transformers_['num'].named_steps['scaler'].get_feature_names_out(numeric_features),
    preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
])

# Get the selected features from the RFE
selected_features = feature_names[pipeline.named_steps['rfe'].support_]
print("Selected Features:", selected_features)


Accuracy: 0.6570247933884298
              precision    recall  f1-score   support

           0       0.69      0.70      0.70       136
           1       0.61      0.60      0.61       106

    accuracy                           0.66       242
   macro avg       0.65      0.65      0.65       242
weighted avg       0.66      0.66      0.66       242



NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.